The 8-puzzle is given as:

Initial state [
[2,8,3],
[1,6,4],
[7,0,5]
]


Goal state [
[1,2,3],
[8,0,4],
[7,6,5]
]

The simple evaluation function f(x) is defined as follows:

f(x) = g(x)+h(x)

where,

g(x) = depth of node X in the search tree (equivalent to cost so far)

h(x) = the number of tiles not in their goal position in a given state X (equivalent to minimum cost to completion)

States: Location of eight tiles plus blank

Initial state: Any state can be designated as the initial state

Actions: Blank tile moves left, right, up, or down and exchanges places with a numbered tile

Successor function: Actions have their expected effects

Goal test: Check whether the goal configuration is reached

Path cost: Each step costs 1

Relied on the code base by https://www.geeksforgeeks.org/8-puzzle-problem-in-ai/

In [15]:
import heapq
import math

# Class to represent the state of the 8-puzzle
class PuzzleState:
    def __init__(self, board, parent, move, depth, cost):
        self.board = board  # The puzzle board configuration
        self.parent = parent  # Parent state
        self.move = move  # Move to reach this state
        self.depth = depth  # Depth in the search tree
        self.cost = cost  # Cost (depth + heuristic)

    def __lt__(self, other):
        return self.cost < other.cost

# Function to display the board
def print_board(board):
    print("+---+---+---+")
    for row in range(0, 9, 3):
        row_visual = "|"
        for tile in board[row:row + 3]:
            if tile == 0:  # Blank tile
                row_visual += f" {(' ')} | "
            else:
                row_visual += f" {(str(tile))} | "
        print(row_visual)
        print("+---+---+---+")

# Goal state for the puzzle
goal_state = [1, 2, 3, 8, 0, 4, 7, 6, 5]

# Possible moves for the blank tile (up, down, left, right)
moves = {
    'U': -3,  # Move up
    'D': 3,   # Move down
    'L': -1,  # Move left
    'R': 1    # Move right
}

# Function to calculate the heuristic (Manhattan distance)
def heuristic(board):
    distance = 0
    for i in range(9):
        if board[i] != 0:
            x1, y1 = divmod(i, 3)
            x2, y2 = divmod(board[i] - 1, 3)
            distance += abs(x1 - x2) + abs(y1 - y2)
    return distance

# Function to get the new state after a move
def move_tile(board, move, blank_pos):
    new_board = board[:]
    new_blank_pos = blank_pos + moves[move]
    new_board[blank_pos], new_board[new_blank_pos] = new_board[new_blank_pos], new_board[blank_pos]
    return new_board

# A* search algorithm
def a_star(start_state):
    open_list = []
    closed_list = set()
    heapq.heappush(open_list, PuzzleState(start_state, None, None, 0, heuristic(start_state)))

    while open_list:
        current_state = heapq.heappop(open_list)

        if current_state.board == goal_state:
            return current_state

        closed_list.add(tuple(current_state.board))

        blank_pos = current_state.board.index(0)

        for move in moves:
            if move == 'U' and blank_pos < 3:  # Invalid move up
                continue
            if move == 'D' and blank_pos > 5:  # Invalid move down
                continue
            if move == 'L' and blank_pos % 3 == 0:  # Invalid move left
                continue
            if move == 'R' and blank_pos % 3 == 2:  # Invalid move right
                continue

            new_board = move_tile(current_state.board, move, blank_pos)

            if tuple(new_board) in closed_list:
                continue

            new_state = PuzzleState(new_board, current_state, move, current_state.depth + 1, current_state.depth + 1 + heuristic(new_board))
            heapq.heappush(open_list, new_state)

    return None

# Function to print the solution path
def print_solution(solution):
    path = []
    current = solution
    while current:
        path.append(current)
        current = current.parent
    path.reverse()

    for step in path:
        print(f"Move: {step.move}")
        print_board(step.board)

# Initial state of the puzzle
initial_state = [2, 8, 3, 1, 6, 4, 7, 0, 5]

# Solve the puzzle using A* algorithm
solution = a_star(initial_state)

# Print the solution
if solution:
    print("Solution found:")
    print_solution(solution)
else:
    print("No solution exists.")

Solution found:
Move: None
+---+---+---+
| 2 |  8 |  3 | 
+---+---+---+
| 1 |  6 |  4 | 
+---+---+---+
| 7 |    |  5 | 
+---+---+---+
Move: U
+---+---+---+
| 2 |  8 |  3 | 
+---+---+---+
| 1 |    |  4 | 
+---+---+---+
| 7 |  6 |  5 | 
+---+---+---+
Move: U
+---+---+---+
| 2 |    |  3 | 
+---+---+---+
| 1 |  8 |  4 | 
+---+---+---+
| 7 |  6 |  5 | 
+---+---+---+
Move: L
+---+---+---+
|   |  2 |  3 | 
+---+---+---+
| 1 |  8 |  4 | 
+---+---+---+
| 7 |  6 |  5 | 
+---+---+---+
Move: D
+---+---+---+
| 1 |  2 |  3 | 
+---+---+---+
|   |  8 |  4 | 
+---+---+---+
| 7 |  6 |  5 | 
+---+---+---+
Move: R
+---+---+---+
| 1 |  2 |  3 | 
+---+---+---+
| 8 |    |  4 | 
+---+---+---+
| 7 |  6 |  5 | 
+---+---+---+
